In [1]:
import subprocess, sys
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "zarr", "numpy", "pandas"], check=False)
print("Dependencies ready.")

Dependencies ready.


# Zarr Cube Verification Notebook

**Project:** Po Valley Rice Resilience Analysis  
**Purpose:** Document the COG → Zarr pipeline for each Sentinel constellation, verify all generated `.zarr` stores, and summarise the results.  
**Date:** 2026-02-27

---

## Section 1 — Workflow Documentation: COG → Zarr per Constellation

The main analysis notebook (`Po_Valley_Resilience_Analysis_Submission 2.ipynb`) follows a unified **cloud-native ingestion & persistence** pattern for all three Sentinel missions.

---

### 1.1 Sentinel-2 (Optical · NDVI)

| Step | Detail |
|---|---|
| **Source** | Earth Search AWS STAC (`sentinel-2-l2a`) |
| **Source Format** | Cloud-Optimized GeoTIFF (COG) — B04 (Red), B08 (NIR), SCL |
| **Ingestion** | `pystac-client` STAC search → `odc-stac.load()` streams COG tiles lazily over HTTP |
| **Processing** | NDVI = (NIR − Red) / (NIR + Red); SCL cloud/shadow pixels → NaN |
| **Persistence** | `xr.Dataset.to_zarr('rice_ndvi_cube_{region}.zarr', mode='w')` |
| **Output Stores** | `rice_ndvi_cube_region_a.zarr`, `rice_ndvi_cube_region_b.zarr` |
| **Data Variable** | `NDVI` — `float32 [time, y, x]`, range −1 to +1 |
| **Zarr Format** | v3 with zstd compression, CRS metadata via `spatial_ref` |

---

### 1.2 Sentinel-1 (Radar · VV Backscatter)

| Step | Detail |
|---|---|
| **Source** | Earth Search AWS STAC (`sentinel-1-grd`) |
| **Source Format** | Cloud-Optimized GeoTIFF (COG) — VV polarisation |
| **Ingestion** | `pystac-client` STAC search → `odc-stac.load()` streams COG tiles lazily over HTTP |
| **Processing** | VV band extracted; subset to region bounding box |
| **Persistence** | `xr.Dataset.to_zarr(zarr_filename, mode='w')` |
| **Output Stores** | `rice_radar_cube_region_a.zarr`, `rice_radar_cube_region_b.zarr` |
| **Data Variable** | `vv` — `uint16 [time, y, x]`, raw DN values (not dB-converted) |
| **Zarr Format** | v3 with zstd compression, CRS metadata via `spatial_ref` |

> **Note on dB:** The `vv` variable stores raw unsigned 16-bit DN values as written by `odc-stac`. The dB conversion (`10 × log₁₀(DN / scale_factor)`) is applied at analysis time, not at write time.

---

### 1.3 Sentinel-3 (Thermal · LST)

| Step | Detail |
|---|---|
| **Source** | Sentinel Hub Process API (CDSE OAuth 2.0) |
| **Source Format** | GeoTIFF (`image/tiff`) — streamed in-memory per scene via HTTP POST |
| **Ingestion** | OAuth token → Sentinel Hub Process API for `sentinel-3-slstr` → raw TIFF bytes read with `rasterio` |
| **Processing** | Per-scene TIFF decoded in-memory; all valid scenes stacked into NumPy cube `[time, y, x]` |
| **Persistence** | `xr.Dataset({'lst': (...)}).to_zarr(zarr_name)` |
| **Output Stores** | `rice_thermal_cube_region_a.zarr`, `rice_thermal_cube_region_b.zarr` |
| **Data Variable** | `lst` — `float64 [time, y, x]`, values in **Kelvin** (215–297 K) |
| **Zarr Format** | v3 with zstd compression; no `spatial_ref` (pixel grid indices only) |

> **Note:** Sentinel-3 is not available as COGs on Earth Search AWS. The Sentinel Hub API returns GeoTIFFs that are decoded in-memory and stacked before saving to Zarr. Values are stored in **Kelvin** (subtract 273.15 to convert to °C).

---

### 1.4 Pipeline Summary

| Constellation | Source | Source Format | Variable | Unit | Output Zarr Stores |
|---|---|---|---|---|---|
| **Sentinel-2** | Earth Search AWS | COG | `NDVI` | dimensionless (−1→1) | `rice_ndvi_cube_region_{a,b}.zarr` |
| **Sentinel-1** | Earth Search AWS | COG | `vv` | raw uint16 DN | `rice_radar_cube_region_{a,b}.zarr` |
| **Sentinel-3** | Sentinel Hub API | GeoTIFF (in-memory) | `lst` | Kelvin | `rice_thermal_cube_region_{a,b}.zarr` |

---

## Section 2 — Verification Code

Each `.zarr` store is verified across six dimensions:

1. **Existence** — does the store directory exist?
2. **Zarr format version** — is `zarr_format` present and valid (2 or 3)?
3. **Expected variable** — is the data variable present?
4. **Non-empty shape** — do all dimensions have size > 0?
5. **Data integrity** — are there any non-zero, non-all-NaN values?
6. **Physical range** — do values fall within expected scientific bounds?

In [2]:
import zarr
import numpy as np
import json, os, warnings
from pathlib import Path

warnings.filterwarnings('ignore')

BASE = Path('.')  # Run from Submission/ directory

# Spec: (display name, zarr path, data variable, expected value range)
# S1 vv: uint16 raw DN — valid range 0–65535; physically near 0–10000 for GRD backscatter
# S3 lst: float64 in Kelvin — land surface temp typically 200–340 K
ZARR_SPECS = [
    {
        'name':          'S2 NDVI – Region A',
        'path':          'rice_ndvi_cube_region_a.zarr',
        'variable':      'NDVI',
        'constellation': 'Sentinel-2',
        'unit':          'dimensionless',
        'value_range':   (-1.0, 1.0),
    },
    {
        'name':          'S2 NDVI – Region B',
        'path':          'rice_ndvi_cube_region_b.zarr',
        'variable':      'NDVI',
        'constellation': 'Sentinel-2',
        'unit':          'dimensionless',
        'value_range':   (-1.0, 1.0),
    },
    {
        'name':          'S1 VV – Region A',
        'path':          'rice_radar_cube_region_a.zarr',
        'variable':      'vv',
        'constellation': 'Sentinel-1',
        'unit':          'uint16 DN',
        'value_range':   (0, 65535),
    },
    {
        'name':          'S1 VV – Region B',
        'path':          'rice_radar_cube_region_b.zarr',
        'variable':      'vv',
        'constellation': 'Sentinel-1',
        'unit':          'uint16 DN',
        'value_range':   (0, 65535),
    },
    {
        'name':          'S3 LST – Region A',
        'path':          'rice_thermal_cube_region_a.zarr',
        'variable':      'lst',
        'constellation': 'Sentinel-3',
        'unit':          'Kelvin',
        'value_range':   (150.0, 340.0),
    },
    {
        'name':          'S3 LST – Region B',
        'path':          'rice_thermal_cube_region_b.zarr',
        'variable':      'lst',
        'constellation': 'Sentinel-3',
        'unit':          'Kelvin',
        'value_range':   (150.0, 340.0),
    },
]

print(f'Loaded specs for {len(ZARR_SPECS)} Zarr stores across 3 constellations.')

Loaded specs for 6 Zarr stores across 3 constellations.


In [3]:
def read_zarr_format(zarr_path):
    """Read zarr_format version from root zarr.json (v3) or .zgroup (v2)."""
    p = Path(zarr_path)
    zj = p / 'zarr.json'
    zg = p / '.zgroup'
    if zj.exists():
        with open(zj) as f:
            return json.load(f).get('zarr_format', 'unknown')
    elif zg.exists():
        return 2
    return 'unknown'


def verify_store(spec):
    """Run all checks on one Zarr store. Returns a result dict."""
    r = {
        'name':          spec['name'],
        'constellation': spec['constellation'],
        'path':          spec['path'],
        'unit':          spec['unit'],
        'exists':        False,
        'zarr_format':   None,
        'openable':      False,
        'has_variable':  False,
        'shape':         None,
        'n_timesteps':   None,
        'all_nan':       None,
        'all_zero':      None,
        'value_min':     None,
        'value_max':     None,
        'in_range':      None,
        'passed':        False,
        'issues':        [],
    }

    path = BASE / spec['path']

    # 1. Existence
    if not path.exists():
        r['issues'].append('Store directory not found')
        return r
    r['exists'] = True

    # 2. Zarr format
    r['zarr_format'] = read_zarr_format(path)
    if r['zarr_format'] not in (2, 3):
        r['issues'].append(f"Unexpected zarr_format: {r['zarr_format']}")

    # 3. Open with zarr (avoids xarray/v3 compat issues)
    try:
        store = zarr.open(str(path), mode='r')
        r['openable'] = True
    except Exception as e:
        r['issues'].append(f'Cannot open store: {e}')
        return r

    # 4. Variable presence
    var = spec['variable']
    if var not in store:
        available = list(store.keys())
        r['issues'].append(f"Variable '{var}' missing. Available: {available}")
        return r
    r['has_variable'] = True

    arr = store[var]

    # 5. Shape
    r['shape'] = arr.shape
    r['n_timesteps'] = arr.shape[0]
    for i, s in enumerate(arr.shape):
        if s == 0:
            r['issues'].append(f'Dimension {i} has size 0')

    # 6. Data integrity — load a small sample (first 5 time steps)
    try:
        sample = arr[0:min(5, arr.shape[0]), :, :].astype(float)
        r['all_nan']  = bool(np.all(np.isnan(sample)))
        r['all_zero'] = bool(np.all(sample == 0))
        valid = sample[~np.isnan(sample)]

        if valid.size == 0:
            r['issues'].append('All sampled values are NaN')
        else:
            r['value_min'] = float(np.min(valid))
            r['value_max'] = float(np.max(valid))
            lo, hi = spec['value_range']
            r['in_range'] = bool(r['value_min'] >= lo and r['value_max'] <= hi)
            if not r['in_range']:
                r['issues'].append(
                    f"Values out of expected range [{lo}, {hi}]: "
                    f"actual=[{r['value_min']:.3f}, {r['value_max']:.3f}]"
                )

        if r['all_zero']:
            r['issues'].append('All sampled values are zero — possible bad write')

    except Exception as e:
        r['issues'].append(f'Error reading data sample: {e}')

    r['passed'] = len(r['issues']) == 0
    return r


print('verify_store() defined. Running verification...')

verify_store() defined. Running verification...


In [4]:
results = []

for spec in ZARR_SPECS:
    print(f'  [{spec["name"]}] ... ', end='', flush=True)
    r = verify_store(spec)
    results.append(r)
    status = '✅ PASS' if r['passed'] else '❌ FAIL'
    print(status)
    if r['issues']:
        for issue in r['issues']:
            print(f'      ⚠  {issue}')

n_pass = sum(r['passed'] for r in results)
print(f'\nResult: {n_pass}/{len(results)} stores passed.')

  [S2 NDVI – Region A] ... 

✅ PASS
  [S2 NDVI – Region B] ... 

✅ PASS
  [S1 VV – Region A] ... 

✅ PASS
  [S1 VV – Region B] ... 

❌ FAIL
      ⚠  All sampled values are zero — possible bad write
  [S3 LST – Region A] ... 

✅ PASS
  [S3 LST – Region B] ... 

✅ PASS

Result: 5/6 stores passed.


---

## Section 3 — Verification Summary

In [5]:
import pandas as pd

rows = []
for r in results:
    t, y, x = (r['shape'] if r['shape'] and len(r['shape'])==3 else (None,None,None))
    rows.append({
        'Store':         r['name'],
        'Constellation': r['constellation'],
        'Zarr v':        r['zarr_format'],
        'Exists':        '✅' if r['exists']      else '❌',
        'Openable':      '✅' if r['openable']     else '❌',
        'Has Var':       '✅' if r['has_variable'] else '❌',
        'Time':          t,
        'Y':             y,
        'X':             x,
        'Unit':          r['unit'],
        'Min':           f"{r['value_min']:.2f}" if r['value_min'] is not None else '—',
        'Max':           f"{r['value_max']:.2f}" if r['value_max'] is not None else '—',
        'In Range':      '✅' if r['in_range'] else ('❌' if r['in_range'] is False else '—'),
        'Result':        '✅ PASS' if r['passed'] else '❌ FAIL',
        'Issues':        '; '.join(r['issues']) if r['issues'] else '—',
    })

df = pd.DataFrame(rows)

pd.set_option('display.max_colwidth', 80)
pd.set_option('display.width', 240)
print('=== Zarr Cube Verification Summary ===')
print(df.to_string(index=False))
print()

total  = len(results)
passed = sum(r['passed'] for r in results)
failed = total - passed
print(f'Stores checked : {total}')
print(f'Passed         : {passed}')
print(f'Failed         : {failed}')

if failed == 0:
    print('\n✅ All Zarr stores are valid and contain data within expected physical bounds.')
else:
    print(f'\n⚠  {failed} store(s) have issues — review the Issues column above.')

# Export
df.to_csv('zarr_verification_summary.csv', index=False)
print('Summary saved → zarr_verification_summary.csv')

=== Zarr Cube Verification Summary ===
             Store Constellation  Zarr v Exists Openable Has Var  Time   Y   X          Unit    Min     Max In Range Result                                           Issues
S2 NDVI – Region A    Sentinel-2       3      ✅        ✅       ✅    56 450 475 dimensionless   0.00    0.98        ✅ ✅ PASS                                                —
S2 NDVI – Region B    Sentinel-2       3      ✅        ✅       ✅    40 234 261 dimensionless   0.00    0.98        ✅ ✅ PASS                                                —
  S1 VV – Region A    Sentinel-1       3      ✅        ✅       ✅   207 450 475     uint16 DN  18.00 6194.00        ✅ ✅ PASS                                                —
  S1 VV – Region B    Sentinel-1       3      ✅        ✅       ✅   239 234 261     uint16 DN   0.00    0.00        ✅ ❌ FAIL All sampled values are zero — possible bad write
 S3 LST – Region A    Sentinel-3       3      ✅        ✅       ✅   243 100 100        Kelvin 255